In [2]:
import sqlite3
import pandas as pd

DB_PATH = 'escola_ingles.db'  # Caminho para o banco de dados SQLite

In [3]:
sql_query = """SELECT * 
FROM cursos
WHERE id_curso == 101
"""

conn = sqlite3.connect(DB_PATH)
df_resultado = pd.read_sql_query(sql_query, conn)
conn.close()

print(df_resultado)

   id_curso       nome_curso        nivel dias_semana  capacidade_maxima  \
0       101  General English  Básico (A1)     Seg/Qua                 20   

   mensalidade  
0        350.0  


In [3]:
sql_query = """
SELECT 
    nome_curso, 
    mensalidade 
FROM 
    cursos
"""

conn = sqlite3.connect(DB_PATH)
df_resultado = pd.read_sql_query(sql_query, conn)
conn.close()

print(df_resultado)

          nome_curso  mensalidade
0    General English        350.0
1    General English        380.0
2   Business English        550.0
3           Kids Fun        320.0
4  Conversation Club        200.0
5    Intensive IELTS        600.0


In [ ]:
sql_query = """
SELECT 
    nome_curso AS Produto, 
    mensalidade AS Ticket_Mensal 
FROM 
    cursos
"""

In [ ]:
def executar_query(sql_query):
    try:
        conn = sqlite3.connect(DB_PATH)
        df_resultado = pd.read_sql_query(sql_query, conn)
        conn.close()
        return df_resultado
    except:
        print("Erro")

In [ ]:
df_retorno = executar_query(sql_query)
print(df_retorno)

          nome_curso  mensalidade
0    General English        350.0
1    General English        380.0
2   Business English        550.0
3           Kids Fun        320.0
4  Conversation Club        200.0
5    Intensive IELTS        600.0


In [ ]:
select_unique = "SELECT DISTINCT dias_semana FROM cursos"
display(executar_query(select_unique))

,dias_semana
0,Seg/Qua
1,Ter/Qui
2,Sáb
3,Sex


In [ ]:
# Query 5: Calculando o valor anual do contrato (Mensalidade x 12)
q5 = """
SELECT 
    nome_curso,
    nivel,
    mensalidade,
    (mensalidade * 12) AS valor_anual_contrato
FROM 
    cursos
"""
display(executar_query(q5))

In [ ]:
import numpy as np
import time

# 1. GERANDO DADOS  (1 Milhão de linhas)
n_rows = 1_000_000
cursos_opcoes = ['Inglês', 'Espanhol', 'Francês', 'Alemão', 'Italiano']

df_big = pd.DataFrame({
    'id_matricula': range(n_rows),
    'curso': np.random.choice(cursos_opcoes, n_rows) # Muitos repetidos
})

# 2. SALVANDO NO BANCO SQLITE
conn = sqlite3.connect('escola_big_data.db')
df_big.to_sql('matriculas_gigantes', conn, if_exists='replace', index=False)
print("✅ Dados salvos no Banco!")

✅ Dados salvos no Banco!


In [ ]:
start = time.time()

# cenário A: Jeito "Pandas" (Ineficiente)
df_local = pd.read_sql("SELECT curso FROM matriculas_gigantes", conn)

# 2. Limpa na memória local
unicos_pandas = df_local['curso'].unique()

end = time.time()
tempo_pandas = end - start
print(f"\n Abordagem Pandas (Trazer tudo + Unique): {tempo_pandas:.4f} segundos")
print(f"-> Memória usada: Tivemos que carregar {len(df_local)} linhas na RAM.")


 Abordagem Pandas (Trazer tudo + Unique): 0.2773 segundos
-> Memória usada: Tivemos que carregar 1000000 linhas na RAM.


In [ ]:
df_local.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 1 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   curso   1000000 non-null  object
dtypes: object(1)
memory usage: 7.6+ MB


In [ ]:
start = time.time()

# 1. O Banco processa e manda só o resultado
query_smart = "SELECT DISTINCT curso FROM matriculas_gigantes"
df_smart = pd.read_sql(query_smart, conn)

end = time.time()
tempo_sql = end - start
print(f"\n Abordagem SQL (SELECT DISTINCT): {tempo_sql:.4f} segundos")
print(f"-> Memória usada: Carregamos apenas {len(df_smart)} linhas na RAM.")

conn.close()


 Abordagem SQL (SELECT DISTINCT): 0.0643 segundos
-> Memória usada: Carregamos apenas 5 linhas na RAM.


In [ ]:
df_smart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   curso   5 non-null      object
dtypes: object(1)
memory usage: 172.0+ bytes


In [ ]:
# Query 6: Top 3 salários/hora
order = """
SELECT 
    nome, 
    cargo, 
    valor_hora_aula 
FROM 
    funcionarios
ORDER BY 
    valor_hora_aula DESC 
LIMIT 3
"""
executar_query(order)

,nome,cargo,valor_hora_aula
0,Sarah Connor,Coordenadora,120.0
1,Pierre Dupont,Professor,95.0
2,Mike Ross,Professor,90.0
